In [11]:
import Pkg;
using Pkg
Pkg.add(url="https://github.com/giordamaug/WiSARDj")
Pkg.add("MLJ")
Pkg.add("MLJBase")
Pkg.add("DataFrames")
Pkg.add("MLBase")
Pkg.add("RDatasets")
Pkg.add("ScikitLearn")
Pkg.add("ScikitLearnBase")
import RDatasets
using MLJ
iris = RDatasets.dataset("datasets", "iris");
first(iris, 3) |> pretty

    Updating git-repo `https://github.com/giordamaug/WiSARDj`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No 

┌─────────────┬────────────┬─────────────┬────────────┬─────────────────────────────────┐
│ SepalLength │ SepalWidth │ PetalLength │ PetalWidth │ Species                         │
│ Float64     │ Float64    │ Float64     │ Float64    │ CategoricalValue{String, UInt8} │
│ Continuous  │ Continuous │ Continuous  │ Continuous │ Multiclass{3}                   │
├─────────────┼────────────┼─────────────┼────────────┼─────────────────────────────────┤
│ 5.1         │ 3.5        │ 1.4         │ 0.2        │ setosa                          │
│ 4.9         │ 3.0        │ 1.4         │ 0.2        │ setosa                          │
│ 4.7         │ 3.2        │ 1.3         │ 0.2        │ setosa                          │
└─────────────┴────────────┴─────────────┴────────────┴─────────────────────────────────┘


In [12]:
using WiSARDj.SciLearnInterface: WiSARDClassifier
using MLJ, MLBase, DataFrames, MLJBase
using ScikitLearn.CrossValidation: cross_val_predict
using ScikitLearnBase

with_cv = true
model = WiSARDClassifier(n_bits=8, n_tics=256, bleaching=false, debug=true)

X = Matrix(DataFrames.select(iris, Not([:Species])))
y = vec(Matrix(DataFrames.select(iris, [:Species])))
if with_cv
    ŷ = cross_val_predict(model, X, y; cv=5)
    y_targets = y
else
    train, test = MLJ.partition(eachindex(y), 0.8, shuffle=true, rng=1)
    ScikitLearnBase.fit!(model, X[train,:], y[train,:])
    ŷ = ScikitLearnBase.predict(model, X[test,:])
    y_targets = y[test,:]
end
accuracy = sum(ŷ .== y_targets) / length(y_targets)
println("accuracy: $accuracy")
MLJBase.ConfusionMatrix()(ŷ, coerce(vec(y_targets), OrderedFactor))  # I don't know why we need coerce!end

accuracy: 0.9466666666666667


┌ Warning: Converting array elements to strings before wrapping in a `CategoricalArray`, as `CategoricalArrays.CategoricalValue{String, UInt8}` unsupported by CategoricalArrays. 
└ @ ScientificTypes /Users/maurizio/.julia/packages/ScientificTypes/3irls/src/convention/coerce.jl:48


              ┌─────────────────────────────────────────┐
              │              Ground Truth               │
┌─────────────┼─────────────┬─────────────┬─────────────┤
│  Predicted  │   setosa    │  versicol…  │  virginica  │
├─────────────┼─────────────┼─────────────┼─────────────┤
│   setosa    │     50      │      0      │      0      │
├─────────────┼─────────────┼─────────────┼─────────────┤
│  versicol…  │      0      │     46      │      4      │
├─────────────┼─────────────┼─────────────┼─────────────┤
│  virginica  │      0      │      4      │     46      │
└─────────────┴─────────────┴─────────────┴─────────────┘
